# Creating Tokens from Text

In [20]:
with open("01_main-chapter-code/the-verdict.txt") as f:
    raw_text = f.read()
print("Total number of characters in the text file: ", len(raw_text))
print(raw_text[:99])

Total number of characters in the text file:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [21]:
import re
result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result = [item for item in result if item.strip()] # Remove space strings
print(result[:30])
print(len(result))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


# Converting Tokens into Token IDs

## Creating Vocabulary

In [37]:
all_words = sorted(set(result))  # Get unique words and sort them
vocab_size = len(all_words)
print("Total number of unique words in the text file: ", vocab_size)

Total number of unique words in the text file:  1130


In [38]:
vocab = {token: integer for integer, token in enumerate(all_words)}  # Create a dictionary of tokens and their IDs
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)
    if i > 10:
        break

('yet', 1125)
('you', 1126)
('younger', 1127)
('your', 1128)
('yourself', 1129)


## Implementing a Simple Text Tokenizer

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocabulary): # We must have a vocabulary to initialize the tokenizer
        self.str_to_int = vocabulary  # Dictionary of tokens and their IDs {"token1": 0, ...}
        self.int_to_str = {v: k for k, v in vocabulary.items()}  # Dictionary of IDs and their tokens {0: "token1", ...}
        
    def encode(self, text):
        # Convert text to a list of token IDs
        preprocessed_text = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed_text = [item.strip() for item in preprocessed_text if item.strip()]
        ids = [self.str_to_int[token] for token in preprocessed_text]
        return ids  # return a list of token IDs
    
    def decode(self, ids):
        # Convert a list of token IDs to text with spaces
        text = " ".join([self.int_to_str[id] for id in ids])
        
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)  # Remove spaces before punctuation
        return text

In [19]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## SimpleTokenizerV1 doesn't handle unknown tokens. Create a new tokenizer SimpleTokenizerV2 that handles unknown tokens.

In [40]:
all_words.extend(["<|endoftext|>", "<|unk|>"])  # Add special tokens for end of text and unknown words
vocab = {token:integer for integer,token in enumerate(all_words)}  # Create a new vocabulary

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1132)
('<|unk|>', 1133)


In [41]:
class SimpleTokenizerV2:
    def __init__(self, vocabulary): # We must have a vocabulary to initialize the tokenizer
        self.str_to_int = vocabulary  # Dictionary of tokens and their IDs {"token1": 0, ...}
        self.int_to_str = {v: k for k, v in vocabulary.items()}  # Dictionary of IDs and their tokens {0: "token1", ...}
        
    def encode(self, text) -> list:
        preprocessed_tokens = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed_tokens = [item.strip() for item in preprocessed_tokens if item.strip()]  # remove empty space strings and empty strings
        preprocessed_tokens = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed_tokens]  # Replace unknown words from source text with <|unk|>
        
        ids = [self.str_to_int[token] for token in preprocessed_tokens]
        return ids
    
    def decode(self, ids) -> str:
        decoded_text = " ".join([self.int_to_str[i] for i in ids])
        decoded_text = re.sub(r'\s+([,.?!"()\'])', r'\1', decoded_text) # Remove spaces before punctuation
        return decoded_text
        

In [43]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [45]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

[1133, 5, 355, 1126, 628, 975, 10, 1132, 55, 988, 956, 984, 722, 988, 1133, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


## A Tokenzier Which can Handle Any Unknown Words: Byte Pair Encoding (BPE)

In [46]:
# Use tiktoken library to implement BPE
import tiktoken
from importlib.metadata import version
print(version("tiktoken"))

0.5.2


In [55]:
tokenizer = tiktoken.get_encoding("gpt2")

In [59]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace. Akwirw ier."
ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(ids)

text = tokenizer.decode(ids)
print(text)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13, 9084, 86, 343, 86, 220, 959, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace. Akwirw ier.


## Data Sampling with a Sliding Window

In [60]:
with open("01_main-chapter-code/the-verdict.txt") as f:
    raw_text = f.read()

tokenizer = tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [61]:
enc_sample = enc_text[50:] # Take a sample of the text

In [62]:
# Create Input/Target Pairs by using a sliding window
context_size = 4
x = enc_sample[:context_size] # Input is the first 4 tokens of sampled text [0:4]
y = enc_sample[1:context_size+1] # Target is the next 4 tokens of sampled text [1:5] (Right shift by 1)
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [66]:
# context and desired are encoded token ids (integers)
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)    

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [70]:
# Do the above loop again with the decoded token ids
for i in range(1, context_size+1):
    context = enc_sample[:i]  # list
    desired = enc_sample[i]  # integer, so need to convert to list
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


# Use a PyTorch Data Loader To Create PyTorch Tensors

In [71]:
import torch
from torch.utils.data import Dataset, DataLoader

In [83]:
from torch import Tensor


# Define the Dataset class
class GPTDataSetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        """
        :param txt: source text
        :param tokenizer: tokenizer object
        :param max_length: number of token ids in a chunk
        :param stride: shift across batches. For example, input/target pair batch 1 is [0:4]/[1:5], batch 2 is [1:5]/[2:6], etc.
        """
        self.input_ids = []  # A list of tensors, or a multi-dimension array
        self.target_ids = [] # A list of tensors, or a multi-dimension array
        
        token_ids = tokenizer.encode(txt)  # Tokenize the text and create token ids
        
        # Create input/target pairs using a sliding window
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx) -> (Tensor, Tensor):
        # Returns a pair of 1D tensors (input_ids, target_ids) at the index idx
        return self.input_ids[idx], self.target_ids[idx]

In [84]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataSetV1(txt, tokenizer, max_length, stride)
    
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

In [85]:
# Use PyTorch DataLoader to get tensors of input/target pairs with length of 4 from raw text
with open("01_main-chapter-code/the-verdict.txt") as f:
    raw_text = f.read()

# batch_size: number of batches in input tensors or target tensors
# max_length: number of token ids in a batch
# stride: shift across batches in input tensors
data_loader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=4, shuffle=False)

data_iter = iter(data_loader) # Iteration of the input/target tensor pairs
print(next(data_iter))

[tensor([[  40,  367, 2885, 1464],
        [1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899]])]


# Creating Embeddings

In [86]:
# Creating a Embedding Layer
torch.manual_seed(123)

vocab_size = 10 # 50,257 words in the BPE tokenizer vocabulary
output_dim = 4 # In GPT-3, the embedding size is 12,288 dimensions

embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880],
        [ 0.3486,  0.6603, -0.2196, -0.3792],
        [ 0.7671, -1.1925,  0.6984, -1.4097],
        [ 0.1794,  1.8951,  0.4954,  0.2692],
        [-0.0770, -1.0205, -0.1690,  0.9178],
        [ 1.5810,  1.3010,  1.2753, -0.2010],
        [ 0.9624,  0.2492, -0.4845, -2.0929],
        [-0.8199, -0.4210, -0.9620,  1.2825],
        [-0.3430, -0.6821, -0.9887, -1.7018],
        [-0.7498, -1.1285,  0.4135,  0.2892]], requires_grad=True)


In [87]:
# For example, we have an input tensor
input_ids = torch.tensor([2, 3, 5, 1])
print(embedding_layer(input_ids))

tensor([[ 0.7671, -1.1925,  0.6984, -1.4097],
        [ 0.1794,  1.8951,  0.4954,  0.2692],
        [ 1.5810,  1.3010,  1.2753, -0.2010],
        [ 0.3486,  0.6603, -0.2196, -0.3792]], grad_fn=<EmbeddingBackward0>)


## Now Create Embeddings for a Text

### Input Token Embeddings

In [92]:
vocab_size = 50257
output_dim = 256

max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)

inputs, targets = next(data_iter)

print("Input Token IDs: \n", inputs)
print("\nInputs shape:\n", inputs.shape)

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print("\nEmbedding Layer:\n", token_embedding_layer)
input_embeddings = token_embedding_layer(inputs)
print("\nInput Embeddings Shape:\n", input_embeddings.shape)

Input Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])

Embedding Layer:
 Embedding(50257, 256)

Input Embeddings Shape:
 torch.Size([8, 4, 256])


### Input Positional Embeddings (Context Embeddings)

In [97]:
context_length = max_length # Positional context has the same number of tokens in a batch
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)  # We only need the same number of positional embeddings as the number of tokens in a batch

pos_embeddings = pos_embedding_layer(torch.arange(context_length))  # torch.arrange creates a tensor with values from 0 to context_length-1
print(pos_embeddings.shape)

torch.Size([4, 256])


## Combine Input Token Embeddings and Positional Embeddings to Input Embeddings

In [98]:
input_embeddings = input_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
